In [ ]:
# Installer les packages nécessaires
!pip install geopandas folium openpyxl ipywidgets jedi

# -------------------------------
# Importations
# -------------------------------
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os
from ipywidgets import FileUpload, Text, FloatText, Dropdown, Button, VBox, HBox, Output
from IPython.display import display
import folium
from google.colab import files
import zipfile

# -------------------------------
# Créer dossier output s'il n'existe pas
# -------------------------------
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# -------------------------------
# Widgets pour upload
# -------------------------------
upload_points = FileUpload(accept=".csv,.xlsx", multiple=False, description="Points Excel/CSV")
upload_territoires = FileUpload(accept=".geojson,.shp", multiple=False, description="Territoires GeoJSON/Shapefile") # Changed description

feuille_excel = Text(value="Data", description="Feuille Excel (si Excel):")
lat_col = Text(value="latitude", description="Colonne Latitude:")
lon_col = Text(value="longitude", description="Colonne Longitude:")

# CRS dropdown d'entrée et sortie
crs_input = Dropdown(
    options=[("EPSG:4326 - WGS 84", "EPSG:4326"),
             ("EPSG:32631 - UTM Zone 31N", "EPSG:32631")],
    value="EPSG:4326",
    description="CRS Entrée:"
)

crs_output = Dropdown(
    options=[("EPSG:4326 - WGS 84", "EPSG:4326"),
             ("EPSG:32631 - UTM Zone 31N", "EPSG:32631")],
    value="EPSG:32631",
    description="CRS Sortie:"
)

tampon_m = FloatText(value=2.0, description="Tampon GPS (m):")
predicate = Dropdown(options=["intersects", "within"], value="intersects", description="Inclure frontières:")

output_csv_name = Text(value="enquetes_territoires.csv", description="Nom du CSV sortie:") # Changed name
output_html_name = Text(value="carte_enquetes.html", description="Nom du HTML sortie:")

btn = Button(description="Exécuter Jointure + Carte")
out = Output()

# -------------------------------
# Fonction de traitement Colab
# -------------------------------
def traiter_jointure_folium_colab(b):
    with out:
        out.clear_output()

        if not upload_points.value or not upload_territoires.value: # Changed variable name
            print("Veuillez uploader les fichiers Points et Territoires.") # Changed message
            return

        points_name = list(upload_points.value.keys())[0]
        territoires_name = list(upload_territoires.value.keys())[0] # Changed variable name

        points_path = f"./{points_name}"
        territoires_path = f"./{territoires_name}" # Changed variable name

        with open(points_path, "wb") as f:
            f.write(upload_points.value[points_name]['content'])
        with open(territoires_path, "wb") as f:
            f.write(upload_territoires.value[territoires_name]['content']) # Changed variable name

        ext = os.path.splitext(points_path)[1].lower()
        try:
            if ext == ".xlsx":
                df = pd.read_excel(points_path, sheet_name=feuille_excel.value)
            elif ext == ".csv":
                df = pd.read_csv(points_path)
            else:
                print("Fichier points non supporté.")
                return
        except Exception as e:
            print("Erreur lecture points:", e)
            return

        if not {lat_col.value, lon_col.value}.issubset(df.columns):
            print(f"Colonnes {lat_col.value} et {lon_col.value} non trouvées.")
            return

        points = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[lon_col.value], df[lat_col.value]),
            crs=crs_input.value
        )

        try:
            territoires = gpd.read_file(territoires_path) # Changed variable name
        except Exception as e:
            print("Erreur lecture territoires:", e) # Changed message
            return

        points = points.to_crs(crs_output.value)
        territoires = territoires.to_crs(crs_output.value) # Changed variable name

        territoires["geometry"] = territoires.buffer(0) # Changed variable name

        if tampon_m.value > 0:
            points["geometry"] = points.geometry.buffer(tampon_m.value)

        # Perform the spatial join, keeping index_right
        resultat = gpd.sjoin(points, territoires, how="left", predicate=predicate.value) # Changed variable name


        if tampon_m.value > 0:
            resultat["geometry"] = resultat["geometry"].centroid

        # Drop the 'Nom_Quartier' column if it exists
        if 'Nom_Quartier' in resultat.columns:
            resultat = resultat.drop(columns=['Nom_Quartier'])

        # Export CSV
        output_csv_path = os.path.join(output_dir, output_csv_name.value)
        # Drop the geometry column before saving to CSV
        resultat_csv = resultat.drop(columns=['geometry'])
        resultat_csv.to_csv(output_csv_path, index=False)


        # Carte Folium
        # Filter points to show only those that found a match (index_right is not null)
        points_wgs = resultat[resultat['index_right'].notna()].to_crs("EPSG:4326")
        territoires_wgs = territoires.to_crs("EPSG:4326") # Changed variable name

        mean_lat = points_wgs.geometry.y.mean() if not points_wgs.empty else 0
        mean_lon = points_wgs.geometry.x.mean() if not points_wgs.empty else 0


        m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)
        folium.GeoJson(territoires_wgs, name="Territoires").add_to(m) # Changed name

        for idx, row in points_wgs.iterrows():
            popup_html = f"<b>Point ID:</b> {idx}<br>"
            # Add all columns from the joined result to the popup except geometry and index_right
            for col in row.index:
                 if col not in ['geometry', 'index_right', lat_col.value, lon_col.value]:
                     popup_html += f"<b>{col}:</b> {row[col]}<br>"


            folium.CircleMarker(
                location=[row[lat_col.value], row[lon_col.value]],
                radius=4,
                color='red',
                fill=True,
                fill_color='red',
                popup=popup_html
            ).add_to(m)

        # Export HTML
        output_html_path = os.path.join(output_dir, output_html_name.value)
        m.save(output_html_path)

        # Créer un ZIP avec les deux fichiers
        zip_path = "resultats_jointure.zip"
        with zipfile.ZipFile(zip_path, "w") as zipf:
            zipf.write(output_csv_path, os.path.basename(output_csv_path))
            zipf.write(output_html_path, os.path.basename(output_html_path))

        print(f"Fichiers regroupés dans : {zip_path}")
        files.download(zip_path)

        display(resultat.head())

# Lier le bouton
btn.on_click(traiter_jointure_folium_colab)

# Affichage widgets
display(VBox([
    upload_points,
    upload_territoires, # Changed variable name
    HBox([feuille_excel, lat_col, lon_col]),
    HBox([crs_input, crs_output, tampon_m, predicate]),
    output_csv_name,
    output_html_name,
    btn,
    out
]))